In [ ]:
'''Office server version Jan 2024
FL for multiple SNRs, updated Apr2023, based on CEL, in progress
class_dir change, test change (use coef_list, also update to stdaln and cel, )
make MergeFL, SplitFL for merge-split

CEL vs Energy detection, 2022 Mar 14 Trying on Colab. 
It seems that processing small batch is slow, 7s per epoch per user
Big batch can accelerate to less than 2s per epoch per user
Copied on 2023Feb3'''
import torch
import os
import math
import torchvision
import torchvision.transforms as transforms
import torch.utils.data.distributed as TUDdistributed
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
import sys
from copy import deepcopy
import random
import collections
#from MobileNetV1_CriticalPath import Net
import matplotlib.pyplot as plt
import pandas as pd 
# from sklearn.externals import joblib
from cnn_models import standalone_cnn
from cnn_models import decouple_cnn
from cnn_models import decouple_cnn_mod

from pytz import timezone
TMZ = timezone('EST')
import datetime
import shutil
import time

device = torch.device("cuda")
use_cuda = True
criterion = nn.CrossEntropyLoss()
random.seed(0) 

def create_net(list, per_class_filter, ty_chs):
    number_class = len(list) # list of locally observable channels
    d_w = number_class*per_class_filter
    cfg = [40, d_w, d_w, d_w ]
    print('corresponding cfg channel list:',cfg)
    net = standalone_cnn( nch=number_class, cfg=cfg, ty_chs=ty_chs)
    net.to(device)
    return net

def testnetsVote(model_list, class_dir, test_loader, coef_list, gain_dif, thresh_sig = 0.5):                             
    '''default for FL, Tests all nodes together, apply majority vote for each band 
    '''
    total = 0 #sum of occupation and emptiness
    test_loss = 0 
    total_ocp = 0 #occupied bands
    total_emp = 0 #empty bands
    correct = 0
    total_cmb = 0  #occupation combination
    correct_ocp = 0
    correct_emp = 0
    correct_cmb = 0
    thresh_logit = -1*(math.log(thresh_sig**(-1) -1))
    # thresh_logit = thresh_sig
    
    with torch.no_grad():
        criterion = nn.BCEWithLogitsLoss()
        for batchidx, (data, target) in enumerate(test_loader):
            target =  target.cuda()
            #print(target.size())
            target = Variable(target)
            target = Variable(target)
#             print('target size is:',target.size())
            output_manual = torch.zeros(target.shape) # manual global decision, float version
            total_batch = float(torch.tensor(target.size()).prod()) #total num of channels in this batch
            total += total_batch
            total_cmb += float(target.size(0))
            total_ocp += float(target.sum())
            total_emp += float(total_batch - target.sum()) 
            if (1-target).sum() != (torch.tensor(target.size()).prod() - target.sum()) : #Testing tensor dim =========
                print('show difference:',(1-target).sum(), total - target.sum())
                print('total ocp/emp calculation wrong')
            '''Above calculate the total number of channels detected and the ground truth number of occupation/emptiness'''
            coef = torch.tensor(coef_list)
            for idx in range(len(class_dir)): # appply fusion strategy on local_node detections
                model_list[idx].eval()
                # model_list[idx](data[idx])
                datain = Variable(gain_dif*1e7*data[idx]).cuda() #1e7 to boost gradient
                localout = (model_list[idx](datain).cpu()>thresh_logit).float()
                output_manual[:,class_dir[idx]]=output_manual[:,class_dir[idx]].add_( localout[:,class_dir[idx]] ) #add local detection result
                # output_manual[:,class_dir[idx]] = output_manual[:,class_dir[idx]]|(model_list[idx](datain).cpu()>thresh_logit)
            output_manual = output_manual.div_( coef ) > 0.49 #get decision fusion: sum observable output and normalize 

            correct_ocp += (output_manual*target.cpu()).sum()
            correct_emp += ((~output_manual)*(1-target.cpu())).sum()
            correct_cmb += (~(output_manual^( target.bool().cpu() ))).float().prod(1).sum() #use XNOR which is 'not+XOR'

            loss = criterion(output_manual.float().cuda(), target)
            test_loss += loss.item()
            # print(output, loss.item())

        correct_tol = deepcopy(correct_ocp) + deepcopy(correct_emp) #just sum
        accuracy_tol = 100*correct_tol/total
        accuracy_pd = 100*correct_ocp/total_ocp
        accuracy_pfa = 100*correct_emp/total_emp
        accuracy_cmb = 100*correct_cmb/total_cmb

    loss = loss.item()
    print('test last batch',(~(output_manual^target.bool().cpu())).float().prod(1).sum() / target.size(0))
    print('++++++++++++++++++ Accuracy on global set: total: %d %%, combination: %d %%, PD: %d %%, PFA: %d %%, loss: %.3f' \
                                            % ( accuracy_tol, accuracy_cmb, accuracy_pd, 100-accuracy_pfa, loss))
    return (accuracy_tol, accuracy_pd, 100-accuracy_pfa)


def testnets4FL(model_list, class_list, test_loader, thresh_sig = 0.5):                             
    '''Tests all nodes together, use TotalDatasetMaker, for FL models but partial_observ
    strategy: any node detects means occupation, only use observ_bands'''
    total = 0 #sum of occupation and emptiness
    test_loss = 0 
    total_ocp = 0 #occupied bands
    total_emp = 0 #empty bands
    correct = 0
    total_cmb = 0  #occupation combination
    correct_ocp = 0
    correct_emp = 0
    correct_cmb = 0
    thresh_logit = -1*(math.log(thresh_sig**(-1) -1))
    # thresh_logit = thresh_sig
    
    with torch.no_grad():
        criterion = nn.BCEWithLogitsLoss()
        for batchidx, (data, target) in enumerate(test_loader):
            target =  target.cuda()
            #print(target.size())
            target = Variable(target)
            target = Variable(target)
#             print('target size is:',target.size())
            output_manual = torch.zeros(target.shape).bool() # manual global decision
            total_batch = float(torch.tensor(target.size()).prod()) #total num of channels in this batch
            total += total_batch
            total_cmb += float(target.size(0))
            total_ocp += float(target.sum())
            total_emp += float(total_batch - target.sum()) 
            if (1-target).sum() != (torch.tensor(target.size()).prod() - target.sum()) : #Testing tensor dim =========
                print('show difference:',(1-target).sum(), total - target.sum())
                print('total ocp/emp calculation wrong')
            '''Above calculate the total number of channels detected and the ground truth number of occupation/emptiness'''

            for idx in range(len(class_dir)): # appply fusion strategy on local detections
                model_list[idx].eval()
                # model_list[idx](data[idx])
                data[idx]= Variable(1e7*data[idx])
                output_manual[:,class_list[idx]] = output_manual[:,class_list[idx]]|(model_list[idx](data[idx].cuda())[:,class_list[idx]].cpu()>thresh_logit)
                # strategy: any node detects means occupation, only use observ_bands
            correct_ocp += (output_manual*target.cpu()).sum()
            correct_emp += ((~output_manual)*(1-target.cpu())).sum()
            correct_cmb += (~(output_manual^( target.bool().cpu() ))).float().prod(1).sum() #use XNOR which is 'not+XOR'

            loss = criterion(output_manual.float().cuda(), target)
            test_loss += loss.item()
            # print(output, loss.item())

        correct_tol = deepcopy(correct_ocp) + deepcopy(correct_emp) #just sum
        accuracy_tol = 100*correct_tol/total
        accuracy_pd = 100*correct_ocp/total_ocp
        accuracy_pfa = 100*correct_emp/total_emp
        accuracy_cmb = 100*correct_cmb/total_cmb

    loss = loss.item()
    print('test last batch',(~(output_manual^target.bool().cpu())).float().prod(1).sum() / target.size(0))
    print('++++++++++++++++++ Accuracy on global set: total: %d %%, combination: %d %%, PD: %d %%, PFA: %d %%, loss: %.3f' \
                                            % ( accuracy_tol, accuracy_cmb, accuracy_pd, 100-accuracy_pfa, loss))
    return (accuracy_tol, accuracy_pd, 100-accuracy_pfa)



def adjust_learning_rate(optimizer, epoch):
    '''Sets the learning rate to the initial LR decayed by 10 every 30 epochs'''
    epoch_inter = 20
    bs = 0.2
    # bs = 1
    lr = 0.05 * bs ** (epoch//epoch_inter)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def train(model, epoch, class_list, train_loader, thresh_sig = 0.5):
    '''Use NodeDatasetMaker by default, trainloader only gives local data and only trains a local model'''
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=3e-4) #decay=0 
    # optimizer = optim.Adam(model.parameters(), lr=0.1, weight_decay=1e-4) #
    # optimizer = optim.AdamW(model.parameters(), lr=0.1, weight_decay=1e-4) #
    adjust_learning_rate(optimizer, epoch)
#     print("\nLocal Epoch", epoch)
    sys.stdout.flush()

    for param_group in optimizer.param_groups:
        print('Learning Rate: %f' % param_group['lr'])
    # sys.stdout.flush()
    model.train()
    thresh_logit = -1*(math.log(thresh_sig**(-1) -1))
#     print('threshold on output logits', thresh_logit)
    train_loss = 0
    criterion = nn.BCEWithLogitsLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)     
        #print( data.size() )
        data, target = Variable(data*1e7), Variable(target)
        optimizer.zero_grad()
        #output = model(data, len(class_list))
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()#remains to be edited
    return model

def gather_layer_FL(model): # Gather layers of a model to 3 categories: convlist, batchnormlist, fclist
    convlist = []
    batchnormlist= []
    
    for layer in model.features:
        if isinstance(layer, nn.Conv2d):
            convlist.append(layer)
        if isinstance(layer, nn.BatchNorm2d) or isinstance(layer, nn.GroupNorm):
            batchnormlist.append(layer)   
    nnfc = model.fc # should be nn.modulelist
    return convlist, batchnormlist, nnfc


def Merge_FL(net_list, net_tol, idx_list, class_dir): 
    '''Fed-Avg Merge, dealing with normal CNN'''
#--------------------------------------create and zero dic_tol-----------------------------------
    net_tol = deepcopy(net_tol)
    dict_tol = net_tol.state_dict()
    for name in list( dict_tol ):  #zero dict_tol
        dict_tol[name].data.zero_()

    net_tol.load_state_dict(dict_tol) #make zero net_tol
    convlist_n0, batchnormlist_n0, fc0  = gather_layer_FL(net_tol)
#----------------------------------------Add node params to net_tol------------------------------------------
    for i in range( len(net_list) ):
        convlist_ni, batchnormlist_ni, fci = gather_layer_FL(net_list[i])
        #---------------------------------------Merge shared layers------------------------
        for [m0, mi] in zip( convlist_n0, convlist_ni ):
            m0.weight.data += mi.weight.data /len(class_dir)
            m0.bias.data += mi.bias.data /len(class_dir) 
        for [m0, mi] in zip( batchnormlist_n0, batchnormlist_ni ):
            m0.weight.data += mi.weight.data /len(class_dir) # batchnormlist has weight?
            m0.bias.data += mi.bias.data /len(class_dir)
            if isinstance(m0, nn.BatchNorm2d):
                m0.running_mean.data += mi.running_mean.data /len(class_dir)
                m0.running_var.data += mi.running_var.data /len(class_dir)
        #-----------------------------------------Merge (Add) FC-------------------------------------------------
        fc0.weight.data += fci.weight.data/ len(class_dir)
        fc0.bias.data += fci.bias.data/ len(class_dir)

    return net_tol    


def Split_FL(net_list, net_tol, idx_list, class_dir):

    convlist_n0, batchnormlist_n0, fclist_n0  = gather_layer_FL(net_tol) # actaully fclist is not list
    list_shared = list( range(shared_layers) )
    for i in range(len(net_list)):
        convlist_ni, batchnormlist_ni, fclist_ni = gather_layer_FL(net_list[i])
        #---------------------------Split Shared Layers-----------------------------------------
        for [m0, mi] in zip( convlist_n0, convlist_ni):       
            #kk += 1
            mi.weight.data = deepcopy(m0.weight.data) # for convs in common, just update             
            mi.bias.data = deepcopy(m0.bias.data) # for convs in common, just update       
        #print('@@@@@=====@@@@@           check itr',k)    
        for [m0, mi] in zip( batchnormlist_n0, batchnormlist_ni):
            mi.weight.data = deepcopy(m0.weight.data)
            mi.bias.data = deepcopy(m0.bias.data)
            if isinstance(m0, nn.BatchNorm2d):
                mi.running_mean.data = deepcopy(m0.running_mean.data)
                mi.running_var.data = deepcopy(m0.running_var.data)      
    #-------------------------Split FC layerrrr ---------------------------------------------
        fclist_ni.weight.data = deepcopy(fclist_n0.weight.data)
        fclist_ni.bias.data = deepcopy(fclist_n0.bias.data)

    return net_list

def Dis_analysis(class_dir, tol_list):
  #idx_list: list of lists, global positions of locally observable bands for each node
  #coef_list: coef for averaging the param for each band, how many nodes are learning each certain band
    idx_list = []
    coef_list = [0]*len( tol_list )
    for i in range( len(class_dir) ): #Generating the mapping btw nodes and net_tol
        sub_idx_list = [] 
        for j in class_dir[i]:
            for k in range( len(tol_list) ):
                if j == tol_list[k]:
                    sub_idx_list.append(k)
                    coef_list[k] += 1 
                    break
        idx_list.append(sub_idx_list)
    return idx_list, coef_list


"""626/720 dataset maker, each logit corresponds to the occupation of a single channel"""
class TotalDatasetMaker(Dataset):
    "simple version that requires the user to edit input/label format elsewhere"
    def __init__(self, db, label_list, transformFunc ):
        """
        db: a list of input signal tensors, label_list: a list of data labels, corresponding to db.
        """
        self.datasets = db
        self.label_list = label_list
        self.transformFunc = transformFunc
    def __getitem__(self, i):
        img = self.datasets[i]
        img = self.transformFunc(img)
        class_label = self.label_list[i]
        return img, class_label

    def __len__(self):
        return len(self.label_list)
    
    
"""720 dataset maker, data looks like: each global channel occupation condition ==> 
each node only learn from 'local' received signal(full size for each node) which is an element in the list of this condition """
class NodeDatasetMakerFL(Dataset):
    '''Node dataset maker, FedAvg version'''
    def __init__(self, db, label_list, node, class_dir, transformFunc ):
        """
        db: a list of input signal tensors, label_list: a list of data labels, corresponding to db.
        node
        """
        self.datasets = db
        self.label_list = label_list
        self.transformFunc = transformFunc
        self.chn_list = class_dir[node]
        self.node = node
    def __getitem__(self, i):
        img = self.datasets[i][self.node]
        img = self.transformFunc(img)
        class_label = self.label_list[i] # For FL, only change here
        return img, class_label

    def __len__(self):
        return len(self.label_list)

def setDir(filepath):
  # if directory not exist, create. if directory already exist, empty it.
  if not os.path.exists(filepath):
    os.makedirs(filepath)
  else:
    print('Directory already exists')
    shutil.rmtree(filepath, ignore_errors = True)
    os.mkdir(filepath)

        
print('start')


In [ ]:
for snr in [12]: #[12, 8 ]: #[12, 8, 14, 16, 20] [4, 6, 8, 10, 12, 14, 16]

    #Load data ===================#===================#===================#===================#===================#===================
    volum = 20
    batchsz=64
    SNR= -1*snr 
    nepoch = 50
    roc_dots = 100
    DistAmp_tr = 10 # DistAmp = 10 #25
    DistAmp_te = 10
    alpha_tr = 3.71
    alpha_te = 3.71#5
    gain_dif= (DistAmp_te*2*3**0.5 /3)**alpha_te / (DistAmp_tr*2*3**0.5 /3)**alpha_tr    
    

    datadir_tr = 'RefinedNewData/SNRs/'+str(DistAmp_tr)+'m/Data_SNR'+str(SNR)+'vol'+str(volum)+'.pth'
    datadir_te = 'RefinedNewData/SNRs/RandMod/Data_SNR'+str(SNR)+'vol1'+'.pth' # RandMod: revision dataset, PU mod varying (testing data only)
    datadir_te = 'RefinedNewData/SNRs/RandMod/Tr20Te5/Data_SNR'+str(SNR)+'vol'+str(volum)+'.pth' # RandMod: revision dataset, PU mod varying (testing data only)
    datadir_tr = datadir_te

    now=datetime.datetime.now(TMZ) #time watermark
    time_watermark = now.strftime('%y%m%d_%H_%M')
    print('model watermark',time_watermark)
    #child dir nbamed by time_watermark
    
    stage_dir='/SNRs/RandModTrTe/'#for naming and directory
    '''address_model: Where to save model:'''
    address_model = 'Saved_Models/'+'FL'+stage_dir+str(SNR)+'dB'+str(volum)+'_'+time_watermark+'/' #root dir for saved models

    data_dict_tr = torch.load(datadir_tr)
    data_dict_te = torch.load(datadir_te)
    data_dict_tr.keys()

    db = data_dict_tr['training data list']
    label_list = data_dict_tr['training label list']

    db_te = data_dict_te['testing data list']
    label_list_te = data_dict_te['testing label list']

    #create models ===================#===================#===================#===================#===================#===================
    per_class_filter = 8
    shared_layers = 100 # for FL, larger than conv layer, 

    class_dir_real=[[0, 3, 4, 19], [0, 1, 10, 4, 19], [1, 10, 4, 19, 5, 13], [1, 10, 2, 11, 14, 5, 13], [2, 11, 14, 5, 13, 6, 15, 17], [3, 4, 19, 7, 12, 18], [4, 19, 7, 12, 18, 8, 16], [4, 19, 5, 13, 8, 16], [5, 13, 8, 16, 9], [5, 13, 6, 15, 17, 9]]
    class_dir = [ list( range(20) ) for i in range(10)] # every local is Full_Observa
    tol_list = []
    for classi in class_dir:
        tol_list += classi
    tol_list = list( set(tol_list) )
    tol_list.sort()

    print(class_dir)
    print(tol_list)

    # get coef 
    idx_list, coef_list = Dis_analysis(class_dir_real, tol_list)
    print(idx_list, coef_list)

    #Parepare nets
    net_list = []
    net_tol = create_net(tol_list, per_class_filter, True) 
    Acc_tol=[]
    Acc_list = []

    for i in range(len(class_dir)):
        net_list.append(create_net(class_dir[i], per_class_filter, True))
        Acc_list.append([])

    db_tr_list = []
    for idx in range(len(class_dir)):# train datasets are more complex
        db_tr_list.append(NodeDatasetMakerFL( db, label_list, idx, class_dir, transforms.Compose([ ]) ))
    db_te_1 = TotalDatasetMaker( db_te, label_list_te, transforms.Compose([ ]) )

    train_loader_list = []
    for idx in range(len(class_dir)):# trainloaders are more complex
        train_loader_list.append(DataLoader(db_tr_list[idx], batch_size=batchsz, shuffle=True, num_workers=8, pin_memory=True))
    # tol_train_loader = DataLoader(tol_trainsets, batch_size=100, shuffle=False, num_workers=0, pin_memory=True)
    tol_test_loader = DataLoader(db_te_1, batch_size=1024, shuffle=False, num_workers=4, pin_memory=True)


    setDir(address_model+'checkpoint/') # if dir not exist, create. if dir already exist, empty it.
    setDir(address_model+'bestmodel/')
    # name1 = type(v).__name__ +'_SNR'+str(SNR)+'vol'+str(volum)+'.pth'
    print('Models saved to dir:\n', address_model)
    name0 = 'FL' +'_SNR'+str(SNR)+'vol'+str(volum) # common part of DNN node names

    txt=open(address_model+'Datasetdir.txt',"w").write(datadir_tr) #save dataset dir (dataset version)

    #training===================#===================#===================#===================#===================#===================
    aggre_inter = 1

    net_tol  = Merge_FL(net_list, net_tol, idx_list, class_dir) #global varaible
    net_list = Split_FL(net_list, net_tol, idx_list, class_dir)

    Acc_PD = []
    Acc_PFA = []
    Acc_cmb = []
    Acc_tol = []
    Acc_bfMerge = [] #acc before merging nets
    plt.title("Global Model ACC of the proposed method")
    best_acc = 0 #acc after merge
    best_acc_bfMerge = 0
    result = testnetsVote( net_list, class_dir, tol_test_loader, coef_list, gain_dif, thresh_sig = 0.5 )
    Acc_tol.append( result[0].item()  )
    Acc_PD.append( result[1].item())
    Acc_PFA.append( 100-result[2].item())
    Acc_bfMerge.append(1*Acc_tol[0])
    for epoch in range(nepoch):
        time_start=time.time()
        # Train & save dicts of n1 n2
        print('epoch:',epoch)
        for i in range(len(net_list)):
            net_list[i] = train(net_list[i], epoch, class_dir[i], train_loader_list[i])     
        print("Aggregation every %d epoch, current_interval %d" %(aggre_inter, (epoch % aggre_inter)))
        result = testnetsVote( net_list, class_dir_real, tol_test_loader, coef_list, gain_dif, thresh_sig = 0.5 )

        # Acc_bfMerge.append( testnets( net_list, class_dir, tol_test_loader )[0].item()  )
        Acc_bfMerge.append( result[0].item())
        Acc_PD.append( result[1].item())
        Acc_PFA.append( 100-result[2].item()) # outputs PFA

        local_state = { # save locally trained models
        'net_dict_list': [net.state_dict() for net in net_list],
        'Acc': Acc_bfMerge[-1],
        'epoch': epoch,
        }
        name1 = address_model+'checkpoint/'+name0+ 'local_nodes'+'.pth'
        torch.save(local_state, name1)

        if Acc_bfMerge[-1] > best_acc_bfMerge: #save locally trained best model
            name1 = address_model+'bestmodel/'+name0+ 'local_nodes'+'.pth'
            torch.save(local_state, name1)
            best_acc_bfMerge = 1*Acc_bfMerge[-1]

        if ((epoch+1) % aggre_inter == 0):
            net_tol  = Merge_FL(net_list, net_tol, idx_list, class_dir) #global varaible

            print('@@@===@@@===@@@===@@@===@@@===@@@===@@@===@@@==== Merged nets')
            #Split nets, test;
            net_list = Split_FL(net_list, net_tol, idx_list, class_dir)
            print('=================================================== Splitted nets')
            Acc_tol.append( testnetsVote( net_list, class_dir_real, tol_test_loader, coef_list, gain_dif, thresh_sig = 0.5 )[0].item()  )
            Merged_state = { # local models and global model after merge
            'net_dict_list': [net.state_dict() for net in net_list],
            'net_tol': net_tol.state_dict(),
            'acc': Acc_tol[-1],
            'epoch': epoch, 
            }
            name1 = address_model+'checkpoint/'+name0+ 'merged_nodes'+'.pth'
            torch.save(Merged_state, name1)

            if Acc_tol[-1] > best_acc:
                name1 = address_model+'bestmodel/'+name0+ 'merged_nodes'+'.pth'
                torch.save(Merged_state, name1)
                Best_acc = 1*Acc_tol[-1]         

        plt.figure(1,figsize=(5, 4), dpi=80)
        l1, = plt.plot( Acc_tol, color='blue', label='Avg Acc/band')
        l2, = plt.plot( Acc_PFA, color='red', label='Acc 4 empty')
        l3, = plt.plot( Acc_PD, color='black', label='Acc 4 busy')
        # l4, = plt.plot( Acc_cmb, color='green', label='Acc 4 Combine')
        plt.title('SNR='+str(SNR)+'dB,'+ 'FL'+ ' Model ACC reaches %.3f %%' %  (max(Acc_bfMerge))  )
        plt.legend(loc='lower right')
        plt.show()

        plt.figure(2,figsize=(5, 4), dpi=80)
        l1, = plt.plot( Acc_tol, color='blue',label='Acc after merge')
        l2, = plt.plot( Acc_bfMerge, color='red', label='Acc before merge')
        plt.legend(loc='lower right')
        plt.title('SNR='+str(SNR)+'dB,'+ 'FL' + ' Model ACC reaches %.3f %%(after merge) and %.3f %%(before merge)' %  (max(Acc_tol), max(Acc_bfMerge)) )
        plt.show()        

        time_end=time.time()
        print('1 epoch time cost:',time_end-time_start,'s')

    df1 = pd.DataFrame()
    # df1['acc_old'] = xx
    df1['Accuracy aft Merg'] = Acc_tol
    df1['Accuracy before Merg'] = Acc_bfMerge
    df1['FA'] = Acc_PFA
    df1['PD'] = Acc_PD
    # print('test loaded best models',testnets4FL( net_list, class_dir, tol_test_loader ))

    with pd.ExcelWriter(address_model + "converg"+"SNR"+str(SNR)+".xlsx", mode='w') as writer:  #mode was 'a'
        df1.to_excel(writer, sheet_name='FL')
    print('statics saved to excel:', address_model + "converg"+"SNR"+str(SNR)+".xlsx")

    '''ROC module of current standalone model, saved in pd2 and pfa2'''
    #ROC ==========================================================================================
    pd2= []
    pfa2 = []

    if best_acc < best_acc_bfMerge: #if bf_merge is better
        local_state = torch.load(address_model+'bestmodel/'+name0+ 'local_nodes'+'.pth')
        for i in range(len(net_list)):
            net_list[i].load_state_dict(local_state['net_dict_list'][i]) 
    else: #if merged is better
        Merged_state = torch.load(address_model+'bestmodel/'+name0+ 'merged_nodes'+'.pth')
        net_tol.load_state_dict( Merged_state['net_tol'])
        net_list = Split_FL(net_list, net_tol, idx_list, class_dir)

    for thresh_val in [ (i+.5) / roc_dots for i in range(roc_dots)]:
        print('threshold:', thresh_val)
        CNNoutput = testnetsVote(net_list, class_dir_real, tol_test_loader, coef_list, gain_dif, thresh_sig=thresh_val ) #
        pd2.append(CNNoutput[1].to(torch.device('cpu')).item())
        pfa2.append(CNNoutput[2].to(torch.device('cpu')).item())

    plt.title("ROC of " +'FL'+ " method in SNR="+str(SNR)+"dB")
    l2, = plt.plot(pfa2, pd2, color='green', label='Transformer')
    plt.legend(loc='lower right')
    plt.show()

    dfroc = pd.DataFrame() # save statics to excel
    # df1['acc_old'] = xx
    
    dfroc['PFA'] = pfa2
    dfroc['PD'] = pd2
    with pd.ExcelWriter(address_model + "ROC_SNR"+str(SNR)+".xlsx", mode='w') as writer:  #mode was 'a'
      dfroc.to_excel(writer, sheet_name='FL')
    print('ROC in Excel saved to:', address_model + "ROC_SNR"+str(SNR)+".xlsx")

    ROC_dict = {
        'pd':pd2,
        'pfa':pfa2,
    }
    torch.save(ROC_dict, address_model+'FL'+'ROC.pth')
    print('ROC in Lists saved to:', address_model+'FL'+'ROC.pth')

In [ ]:
from torchsummary import summary 
# v = AlexNet1D(num_classes = 10).to(device)
summary(net_list[0], (1,64,20))

In [ ]:
import gc
gc.collect()
